In [11]:
# coding:utf-8
import urllib3
from bs4 import BeautifulSoup
import certifi
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_binary
import csv

# アクセスするURL
vol_num = 15
issue_num = 1
while vol_num < 29:
    root_url = "https://www.sciencedirect.com"
    url = root_url + "/journal/cirp-journal-of-manufacturing-science-and-technology/vol/"  + str(vol_num) 

    # 書き込み用CSV
    csv_file = "C:\\Users\<username>\Documents\JournalList\journal_list_vol" + str(vol_num) + ".csv"

    # ブラウザーを起動
    options = Options()

    options.binary_location = '/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary'
    options.add_argument('--headless')
    driver = webdriver.Chrome(r"C:\Users\saori\Anaconda3\Lib\site-packages\chromedriver_binary\chromedriver.exe")

    # httpsの証明書検証を実行している
    http = urllib3.PoolManager(
        cert_reqs='CERT_REQUIRED',
        ca_certs=certifi.where())
    r = http.request('GET', url)

    soup = BeautifulSoup(r.data, 'html.parser')

    # 初期化
    header_data = []
    header_author_data = []
    author_list_append = []

    # 論文リストから要素を抽出
    list_all = soup.find_all("li", class_="js-article-list-item article-item u-padding-xs-top u-margin-l-bottom")

    #各論文ページを参照
    for list_row in list_all :

        # 初期化
        inst_searchlist = []

        additional_url = list_row.find("a", attrs={"class", "anchor article-content-title u-margin-xs-top u-margin-s-bottom"}).get('href')
        url_child = root_url + additional_url


        #Beautiful Soup
        r_child = http.request('GET', url_child)
        child_data = BeautifulSoup(r_child.data, 'html.parser')

        title = child_data.find("span", attrs={"class", "title-text"}).string


        #Web Driver
        driver.get(url_child)

        #Show Detailsを開ける
        sleep(3)
        driver.find_element_by_class_name("show-hide-details.u-font-sans").click()

        #Show Detailsを開けた後の状態で取得しなおす
        sleep(1)

        inst_list = driver.find_elements_by_class_name("affiliation")    
        author_list = child_data.find_all("a", attrs={"class", "author size-m workspace-trigger"})

        # 所属機関取得
        inst_list_len = len(inst_list)

        if inst_list_len > 1:

            for an_inst in inst_list :     

                # 複数人著者がいるのに、一人ごとに所属機関が記載されている場合の対策                 
                try: 
                    an_inst.find_element_by_tag_name("sup")
                except:
                    inst_ref = None
                else:
                    inst_ref = an_inst.find_element_by_tag_name("sup").text
                    
                inst_name = an_inst.find_element_by_tag_name("dd").text

                # 所属機関が複数ある場合のみinst_listを使う
                inst_searchlist.append([inst_ref, inst_name])

        elif inst_list_len == 1:
            for an_inst in inst_list :    
                inst_ref = None            
                inst_name = an_inst.find_element_by_tag_name("dd").text

        else:
            inst_ref = None
            inst_name = None



        # 著者リスト取得
        for an_author in author_list :

            given_name = an_author.find("span", attrs={"class", "text given-name"}).string
            surname = an_author.find("span", attrs={"class", "text surname"}).string
            name = given_name + surname

            author_ref = an_author.find("span", attrs={"class", "author-ref"})

            # 著者名の右にレファレンス番号（aなど）があった場合、所属機関検索リストから検索
            if author_ref is not None:
                author_ref = author_ref.string
                for each in inst_searchlist:
                    if author_ref == each[0]:
                        inst_name = each[1]
                        break
                    else:
                        continue

            author_list_append.append([vol_num, issue_num, title, url_child, name, author_ref, inst_name])


    with open(csv_file, 'w', encoding='UTF-8', newline="") as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL)
        writer.writerows(author_list_append)

    vol_num = vol_num + 1     
    
#     if issue_num == 4:
#         vol_num = vol_num + 1
#         issue_num = 1
#     else:
#         issue_num = issue_num + 1
    
    print("issue finished!")
print("end all available issues!")

issue finished!
issue finished!
issue finished!
issue finished!
issue finished!
issue finished!
issue finished!
issue finished!
issue finished!
issue finished!
issue finished!
issue finished!
issue finished!
issue finished!
end all available issues!


finished!
